In [1]:
# 一，单例的实现和普通类的状态共享

In [15]:
# 普通类的状态共享
class brogA(object):
    """ 非单例类，可在类之间共享状态"""
    __shared_state = {}
    def __init__(self):
        self.__dict__ = self.__shared_state



In [20]:
class sign_brogA(brogA):
    
    def __init__(self):
        brogA.__init__(self)   # 此步确保状态共享
        self.sta = 'init'
    def __str__(self):
        return self.sta

In [21]:
sa1 = sign_brogA()
sa2 = sign_brogA()
sa1.sta = 'abc'   # 给sa1的属性赋值,sa2 也会有
print(sa1)
print(sa2)

abc
abc


In [22]:
sa1.x = 'xx1'   # 给sa1 添加新属性，sa2 实例也会有
print(sa1.x)
print(sa2.x)

xx1
xx1


In [23]:
# new单例，需要额外开销来维护子类的单例状态
class singleton(object):
    _instance = None
    def __new__(cls):        # __new__实现单例
        if cls._instance == None:     
            cls._instance = object.__new__(cls)
        return cls._instance

# 测试 创建多个类实例，内存中始终只有一个
def test_sing(cls):
    s1 = singleton()
    s2=singleton()
    return s1==s2
# 测试2:子类继承
class singA(singleton):
    pass

print(test_sing(singA))

True


In [26]:
# 元类 type 单例
class MetaSingleton(type):
    """ A type for Singleton classes (overrides __call__) """    

    def __init__(cls, *args):
        print(cls,"__init__ method called with args", args)
        type.__init__(cls, *args)
        cls.instance = None

    def __call__(cls, *args, **kwargs):
        if not cls.instance:
            print(cls,"creating instance", args, kwargs)
            cls.instance = type.__call__(cls, *args, **kwargs)
        return cls.instance

class Singleton(object):
    """ Singleton in Python """
    
    _instance = None
    
    def __new__(cls):
        if cls._instance == None:
            cls._instance = object.__new__(cls)
        return cls._instance

class SingletonA(Singleton):
    pass

class SingletonA1(SingletonA):
    pass

class SingletonB(Singleton):
    pass

class SingletonM(metaclass=MetaSingleton):
    pass

def test_single(cls):
    """ Test if passed class is a singleton
    测试方法"""
    return cls() == cls()

<class '__main__.SingletonM'> __init__ method called with args ('SingletonM', (), {'__module__': '__main__', '__qualname__': 'SingletonM'})


In [27]:
a = SingletonA()
a1 = SingletonA1()
b = SingletonB()

a.x = 100
print('a.x =>',a.x)
print('a1.x =>',a1.x)
# Will raise an exception
try:
    print('b.x =>',b.x)
except AttributeError as e:
    print('Error:',e)

print(test_single(Singleton))
print(test_single(SingletonM))
print(test_single(SingletonA))
print(test_single(SingletonB))
print(test_single(SingletonA1))

a.x => 100
a1.x => 100
Error: 'SingletonB' object has no attribute 'x'
True
<class '__main__.SingletonM'> creating instance () {}
True
True
True
True


In [29]:
# 二，工厂模式类的实现
from abc import ABCMeta, abstractmethod

class Employee(metaclass=ABCMeta):
    """ An Employee class """

    def __init__(self, name, age, gender):
        self.name = name
        self.age = age
        self.gender = gender

    @abstractmethod
    def get_role(self):
        pass
    
    def __str__(self):
        return "{} - {}, {} years old {}".format(self.__class__.__name__,
                                                 self.name,
                                                 self.age,
                                                 self.gender)
class Engineer(Employee):
    """ An Engineer Employee """
    
    def get_role(self):
        return "engineering"

class Accountant(Employee):
    """ An Accountant Employee """
    
    def get_role(self):
        return "accountant"    

class Admin(Employee):
    """ An Admin Employee """

    def get_role(self):
        return "administration"

In [36]:
class EmployeeFactory(object):
    """ An Employee factory class 
    工厂类，通过一个入口创建多个类
    特征: 通常使用一个工厂类关联整个类家族，类及子类的层次结构"""

    @classmethod
    def create(cls, name, *args):
        """ Factory method for creating an Employee instance 
       特征: 工厂类的方法通常为 classmethod声明，这样可以通过类的命名空间直接调用，可以不用创建实例
       如EmployeeFactory.create('engineer', 'Sam', 25, 'M')"""

        name = name.lower().strip()
        
        if name == 'engineer':
            return Engineer(*args)
        elif name == 'software engineer':
            return SoftwareEngineer(*args)
        elif name == 'admin':
            return Admin(*args)
        elif name == 'accountant':
            return Accountant(*args)

In [40]:
factory = EmployeeFactory()
print(factory.create('engineer', 'Sam', 25, 'M'))
print(factory.create('accountant', 'Ham', 36, 'F'))

Engineer - Sam, 25 years old M
Accountant - Ham, 36 years old F


In [38]:
print(factory.create('admin', 'Jam', 26, 'F'))

Admin - Jam, 26 years old F


In [42]:
# 三，迭代器模式类的实现
import itertools

class Prime(object):
    """ An iterator for prime numbers 
    无限迭代器，生成指定两个数之间的质数序列"""

    def __init__(self, initial, final=0):
        """ Initializer - accepts a number """
        # This may or may not be prime
        self.current = initial
        self.final = final
        
    def __iter__(self):
        return self

    def __next__(self):
        """ Return next item in iterator """
        return self._compute()

    def _compute(self):
        """ Compute the next prime number """

        num = self.current
        
        while True:
            is_prime = True
            
            # Check this number
            for x in range(2, int(pow(self.current, 0.5)+1)):
                if self.current%x==0:
                    is_prime = False
                    break


            num = self.current
            self.current += 1

            if is_prime:
                return num
            
            # If there is an end range, look for it
            if self.final>0 and self.current>self.final:
                raise StopIteration

In [44]:
print('2-50之间的质数:',list(Prime(2,50)))
# First 100 primes
print('前100个质数:',list(itertools.islice(Prime(2), 100)))
# First 10 primes ending with 1
print('# 个位以1结尾的质数:',list(itertools.islice(itertools.filterfalse(lambda x: x % 10 != 1, Prime(2)), 10))) # 个位以1结尾的质数
# First 10 palindromic primes
print('前十个回文质数:',list(itertools.islice(itertools.filterfalse(lambda x: str(x)!=str(x)[-1::-1], Prime(10)), 10)))  # 前十个回文质数

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47]
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541]
# 个位以1结尾的质数: [11, 31, 41, 61, 71, 101, 131, 151, 181, 191]
前十个回文质数: [11, 101, 131, 151, 181, 191, 313, 353, 373, 383]


In [45]:
# 四，结构化-适配器模式类的实现

import itertools

class Polygon(object):
    """ A polygon class """
    
    def __init__(self, *sides):
        """ Initializer - accepts length of sides """
        self.sides = sides
        
    def perimeter(self):
        """ Return perimeter """
        
        return sum(self.sides)
    
    def is_valid(self):
        """ Is this a valid polygon """
        
        # Do some complex stuff - not implemented in base class
        raise NotImplementedError
    
    def is_regular(self):
        """ Is a regular polygon ? """
        
        # Yes: if all sides are equal
        side = self.sides[0]
        return all([x==side for x in self.sides[1:]])
    
    def area(self):
        """ Calculate and return area """
        
        # Not implemented in base class
        raise NotImplementedError

class InvalidPolygonError(Exception):
    pass


In [46]:
class Triangle(Polygon):
    """ Triangle class from Polygon using class adapter """
    
    def is_equilateral(self):
        """ Is this an equilateral triangle ? """
        
        if self.is_valid():
            return super(Triangle, self).is_regular()
    
    def is_isosceles(self):
        """ Is the triangle isosceles """
        
        if self.is_valid():
            # Check if any 2 sides are equal
            for a,b in itertools.combinations(self.sides, 2):
                if a == b:
                    return True
        return False
    
    def area(self):
        """ Calculate area """
        
        # Using Heron's formula
        p = self.perimeter()/2.0
        total = p
        for side in self.sides:
            total *= abs(p-side)
            
        return pow(total, 0.5)
    
    def is_valid(self):
        """ Is the triangle valid """
        
        # Sum of 2 sides should be > 3rd side
        perimeter = self.perimeter()
        for side in self.sides:
            sum_two = perimeter - side
            if sum_two <= side:
                raise InvalidPolygonError(str(self.__class__) + "is invalid!")
                
        return True

class Rectangle(Polygon):
    """ Rectangle class from Polygon using class adapter """

    def is_square(self):
        """ Return if I am a square """

        if self.is_valid():
            # Defaults to is_regular
            return self.is_regular()

    def is_valid(self):
        """ Is the rectangle valid """

        # Should have 4 sides
        if len(self.sides) != 4:
            return False

        # Opposite sides should be same
        for a,b in [(0,2),(1,3)]:
            if self.sides[a] != self.sides[b]:
                return False

        return True

    def area(self):
        """ Return area of rectangle """

        # Length x breadth
        if self.is_valid():
            return self.sides[0]*self.sides[1]

In [49]:
t1 = Triangle(2,2,2)
print(t1.is_equilateral())
print(t1.is_isosceles())

t2 = Triangle(2,2,3)
print(t2.is_equilateral())
print(t2.is_isosceles())

True
True
False
True


In [54]:
# 五，结构化-外观模式
# 通过一些仪表盘和控件，工程师就能挑战控制系统背后复杂系统，保持系统运行，隐藏系统背后的复杂性
# compiler 模块词法分析器，解析器，ast树生成器等的外观
import time

class Engine(object):
    """ An Engine class
    发动机"""
    
    def __init__(self, name, bhp, rpm, volume, cylinders=4, type='petrol'):
        self.name = name
        self.bhp = bhp
        self.rpm = rpm
        self.volume = volume
        self.cylinders = cylinders
        self.type = type

    def start(self):
        """ Start-up the engine """
        print('Engine started')

    def stop(self):
        """ Stop the engine """
        print('Engine stopped')

In [55]:
class Transmission(object):
    """ Transmission class 
    转动控制"""

    def __init__(self, gears, torque):
        self.gears = gears
        self.torque = torque
        # Start with neutral
        self.gear_pos = 0

    def shift_up(self):
        """ Shift up gears """

        if self.gear_pos == self.gears:
            print('Cant shift up anymore')
        else:
            self.gear_pos += 1
            print('Shifted up to gear',self.gear_pos)

    def shift_down(self):
        """ Shift down gears """

        if self.gear_pos == -1:
            print("In reverse, can't shift down")
        else:
            self.gear_pos -= 1
            print('Shifted down to gear',self.gear_pos)         

    def shift_reverse(self):
        """ Shift in reverse """

        print('Reverse shifting')
        self.gear_pos = -1

    def shift_to(self, gear):
        """ Shift to a gear position """

        self.gear_pos = gear
        print('Shifted to gear',self.gear_pos)      


In [56]:
# 其他子系统，如刹车，泊车，轮子，悬架，框架等
class Brake(object):
    """ A brake class
    刹车"""

    def __init__(self, number, type='disc'):
        self.type = type
        self.number = number

    def engage(self):
        """ Engage the break """

        print('%s %d engaged' % (self.__class__.__name__,
                                 self.number))

    def release(self):
        """ Release the break """

        print('%s %d released' % (self.__class__.__name__,
                                  self.number))

class ParkingBrake(Brake):
    """ A parking brake class """

    def __init__(self, type='drum'):
        super(ParkingBrake, self).__init__(type=type, number=1)
        

class Suspension(object):
    """ A suspension class """
    
    def __init__(self, load, type='mcpherson'):
        self.type = type
        self.load = load

class Wheel(object):
    """ A wheel class """

    def __init__(self, material, diameter, pitch):
        self.material = material
        self.diameter = diameter
        self.pitch = pitch
                 
class WheelAssembly(object):
    """ A wheel assembly class """
    
    def __init__(self, brake, suspension):
        self.brake = brake
        self.suspension = suspension
        self.wheels = Wheel('alloy', 'M12',1.25)

    def apply_brakes(self):
        """ Apply brakes """

        print('Applying brakes')
        self.brake.engage()

class Frame(object):
    """ A frame class for an automobile """
    
    def __init__(self, length, width):
        self.length = length
        self.width = width

In [57]:
# Car 类 组合子系统并提供两个方法的外观，start 和 stop
class Car(object):
    """ A car class - Facade pattern """

    def __init__(self, model, manufacturer):
        self.engine = Engine('Maruti K-series',85,5000, 1.3)
        self.frame = Frame(385, 170)
        self.wheel_assemblies = []
        for i in range(4):
            self.wheel_assemblies.append(WheelAssembly(Brake(i+1), Suspension(1000)))
            
        self.transmission = Transmission(5, 115)
        self.model = model
        self.manufacturer = manufacturer
        self.park_brake = ParkingBrake()
        # Ignition engaged
        self.ignition = False

    def start(self):
        """ Start the car """

        print('Starting the car')
        self.ignition = True
        self.park_brake.release()
        self.engine.start()
        self.transmission.shift_up()
        print('Car started.')

    def stop(self):
        """ Stop the car """

        print('Stopping the car')
        # Apply brakes to reduce speed
        for wheel_a in self.wheel_assemblies:
            wheel_a.apply_brakes()

        # Move to 2nd gear and then 1st
        self.transmission.shift_to(2)
        self.transmission.shift_to(1)
        self.engine.stop()
        # Shift to neutral
        self.transmission.shift_to(0)
        # Engage parking brake
        self.park_brake.engage()
        print('Car stopped.')

In [58]:
car_jili = Car('Swift', 'Suzuki')
print(car_jili)

In [59]:
car_jili.start()  # 启动
car_jili.stop()  # 停止

Starting the car
ParkingBrake 1 released
Engine started
Shifted up to gear 1
Car started.
Stopping the car
Applying brakes
Brake 1 engaged
Applying brakes
Brake 2 engaged
Applying brakes
Brake 3 engaged
Applying brakes
Brake 4 engaged
Shifted to gear 2
Shifted to gear 1
Engine stopped
Shifted to gear 0
ParkingBrake 1 engaged
Car stopped.


In [61]:
# 六，代理模式
# 目标: 代理总是实现代理对象的接口
# 适用场景
# 
# 1，需要一个更接近客户的虚拟资源，它可以替代另一个网络中的实际资源，如远程代理
# 2，需要控制/监视对资源的访问时，如网络代理和实例计数代理
# 3，需要保护资源或对象(保护代理), 因为直接访问资源会导致安全问题或危机资源，例如反向代理服务
# 4，需要从开销大的计算或网络操作中优化对结果的访问，以便不必每次都指向计算，如缓存代理
from abc import ABCMeta, abstractmethod

class Employee(metaclass=ABCMeta):
    """ An Employee class """

    def __init__(self, name, age, gender):
        self.name = name
        self.age = age
        self.gender = gender

    @abstractmethod
    def get_role(self):
        pass
    
    def __str__(self):
        return "{} - {}, {} years old {}".format(self.__class__.__name__,
                                                 self.name,
                                                 self.age,
                                                 self.gender)
class Engineer(Employee):
    """ An Engineer Employee """
    
    def get_role(self):
        return "engineering"

class Admin(Employee):
    """ An Admin Employee """

    def get_role(self):
        return "administration"

class Accountant(Employee):
    """ An Accountant Employee """

    def get_role(self):
        return "accounting"
    

In [62]:
class EmployeeProxy(object):
    """ Counting proxy class for Employees 
    实现了一个组合或对象代理，包装目标对象"""

    # Count of employees
    count = 0

    def __new__(cls, *args):
        """ Overloaded __new__ """
        # To keep track of counts
        instance = object.__new__(cls)
        cls.incr_count()   # 实例计数
        return instance
        
    def __init__(self, employee):
        self.employee = employee

    @classmethod
    def incr_count(cls):
        """ Increment employee count """
        cls.count += 1

    @classmethod
    def decr_count(cls):
        """ Decrement employee count """
        cls.count -= 1

    @classmethod
    def get_count(cls):
        """ Get employee count """
        print('代理计数,引用的是类计数')
        return cls.count
    
    def __str__(self):
        return str(self.employee)
    
    def __getattr__(self, name):
        """ Redirect attributes to employee instance
        重定向对象访问权"""

        return getattr(self.employee, name)
        
    def __del__(self):
        """ Overloaded __del__ method """
        # Decrement employee count
        self.decr_count()     # 实例计数


In [63]:
class EmployeeProxyFactory(object):
    """ An Employee factory class returning proxy objects """

    @classmethod
    def create(cls, name, *args):
        """ Factory method for creating an Employee instance """

        name = name.lower().strip()
        
        if name == 'engineer':
            return EmployeeProxy(Engineer(*args))
        elif name == 'accountant':
            return EmployeeProxy(Accountant(*args))
        elif name == 'admin':
            return EmployeeProxy(Admin(*args))

In [71]:
Factory = EmployeeProxyFactory()
engineer = Factory.create('engineer', 'Sam','29', 'M')
print(engineer)

Engineer - Sam, 29 years old M


In [73]:
print(engineer.get_count())  # 
print(EmployeeProxy.get_count())  # 
print(Factory.get_count())
del engineer
print(EmployeeProxy.get_count())  # 
print(engineer.get_count())  # 

NameError: name 'engineer' is not defined

In [74]:
a = ['world', [1,2,3,4]]
b = a[:]

In [75]:
print(b)

['world', [1, 2, 3, 4]]


In [76]:
a[0] = 'hello'
a[1].append(5)
print(a)
print(b)

['hello', [1, 2, 3, 4, 5]]
['world', [1, 2, 3, 4, 5]]


In [77]:
c = a.copy()
print(c)

['hello', [1, 2, 3, 4, 5]]


In [79]:
a[0] = 'hi'
a[1].pop(-1)
print(a)
print(c)

['hi', [1, 2, 3, 4]]
['hello', [1, 2, 3, 4]]


In [1]:
a=3
b=2
lambda(a*b, a:b)

SyntaxError: invalid syntax (<ipython-input-1-d6edcd18ca1c>, line 3)

In [32]:
import copy


def gen_list(lis):
    rst_list = []
    rst_list.append(lis)
    for i in list(range(len(lis))):
        other_lis = copy.deepcopy(lis)
        if other_lis[i] > 0:
            for j in list(range(other_lis[i])):
                j_lis = copy.deepcopy(lis)
                j_lis[i] = j
                o_lis = [[j_lis[i], x] for x in list(range(other_lis[-1]))]  # 列表推导式

                print('j_lis:{}'.format(j_lis))
                if j_lis not in rst_list:
                    rst_list.append(j_lis)
                if o_lis not in rst_list:
                    rst_list.extend(o_lis)

                print('now rst_list:{}'.format(rst_list))
        else:
            rst_list.append(other_lis)
    rsts_list = []
    for ls in rst_list:
        rsts_list.append(str(ls))
    rst_list = set(rsts_list) # 去重
    print('rst_list:{}'.format(rst_list))
    # rsts_list = []
    # for lss in rst_list:
    #     rsts_list.append(list(lss[1:-1]))
    # rst_list = rsts_list
    print('rst_len:{}'.format(len(rst_list)))
    return rst_list


In [33]:
print(gen_list([5,4]))

j_lis:[0, 4]
now rst_list:[[5, 4], [0, 4], [0, 0], [0, 1], [0, 2], [0, 3]]
j_lis:[1, 4]
now rst_list:[[5, 4], [0, 4], [0, 0], [0, 1], [0, 2], [0, 3], [1, 4], [1, 0], [1, 1], [1, 2], [1, 3]]
j_lis:[2, 4]
now rst_list:[[5, 4], [0, 4], [0, 0], [0, 1], [0, 2], [0, 3], [1, 4], [1, 0], [1, 1], [1, 2], [1, 3], [2, 4], [2, 0], [2, 1], [2, 2], [2, 3]]
j_lis:[3, 4]
now rst_list:[[5, 4], [0, 4], [0, 0], [0, 1], [0, 2], [0, 3], [1, 4], [1, 0], [1, 1], [1, 2], [1, 3], [2, 4], [2, 0], [2, 1], [2, 2], [2, 3], [3, 4], [3, 0], [3, 1], [3, 2], [3, 3]]
j_lis:[4, 4]
now rst_list:[[5, 4], [0, 4], [0, 0], [0, 1], [0, 2], [0, 3], [1, 4], [1, 0], [1, 1], [1, 2], [1, 3], [2, 4], [2, 0], [2, 1], [2, 2], [2, 3], [3, 4], [3, 0], [3, 1], [3, 2], [3, 3], [4, 4], [4, 0], [4, 1], [4, 2], [4, 3]]
j_lis:[5, 0]
now rst_list:[[5, 4], [0, 4], [0, 0], [0, 1], [0, 2], [0, 3], [1, 4], [1, 0], [1, 1], [1, 2], [1, 3], [2, 4], [2, 0], [2, 1], [2, 2], [2, 3], [3, 4], [3, 0], [3, 1], [3, 2], [3, 3], [4, 4], [4, 0], [4, 1], [4, 2]

In [26]:
[[0, i]  for i in list(range(10))]

[[0, 0],
 [0, 1],
 [0, 2],
 [0, 3],
 [0, 4],
 [0, 5],
 [0, 6],
 [0, 7],
 [0, 8],
 [0, 9]]

TypeError: unhashable type: 'list'